#  **CNN from Scratch on MNIST (NumPy) + PyTorch Comparison**


## 1. Importing Required Libraries
This code cell imports the essential libraries required for the project.
- `numpy` for numerical operations, used in the scratch implementation.
- `time` for measuring execution time.
- `torch`, `torch.nn`, `torch.nn.functional`, `torch.optim` for building and training the PyTorch model.
- `torchvision.datasets` and `torchvision.transforms` for handling the MNIST dataset.

In [ ]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

## Dataset Preparation Introduction
This markdown cell explains that the `torchvision.datasets.MNIST` will be used with transformations to prepare the dataset.

## Loading the MNIST Training Data
This code cell loads the MNIST training dataset using `torch.utils.data.DataLoader`.
- It specifies the dataset location (`../mnist_data`).
- `download=True` ensures the dataset is downloaded if not already present.
- `train=True` specifies that this is the training set.
- `transforms.Compose` applies a sequence of transformations:
    - `transforms.ToTensor()` converts the images to PyTorch tensors and scales pixel values to [0, 1].
    - `transforms.Normalize((0.1307,), (0.3081,))` normalizes the image pixel values using the mean and standard deviation of the MNIST dataset.
- `batch_size=10` sets the batch size for loading data.
- `shuffle=True` shuffles the data at the beginning of each epoch.
The output shows the download progress of the dataset files.

In [ ]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST('../mnist_data',
                                                          download=True,
                                                          train=True,
                                                          transform=transforms.Compose([
                                                              transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                              transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                          ])),
                                           batch_size=10,
                                           shuffle=True)


100%|██████████| 9.91M/9.91M [00:00<00:00, 16.0MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 493kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.83MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.62MB/s]


## Loading the MNIST Test Data
This code cell loads the MNIST test dataset, similar to the training data loading in the previous cell, but with `train=False`.

In [ ]:
test_loader = torch.utils.data.DataLoader(datasets.MNIST('../mnist_data',
                                                          download=True,
                                                          train=False,
                                                          transform=transforms.Compose([
                                                              transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                              transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                          ])),
                                           batch_size=10,
                                           shuffle=True)

##  Converting Training Data to NumPy Arrays
This code cell iterates through the `train_loader` and converts the PyTorch image and label tensors to NumPy arrays, storing them in `X_train_list` and `Y_train_list` respectively. This is done to prepare the data for the NumPy-based CNN implementation.

In [ ]:
X_train_list = []
Y_train_list = []
for images, labels in train_loader:
    X_train_list.append(images.numpy())
    Y_train_list.append(labels.numpy())

## Concatenating Training Data NumPy Arrays
This code cell concatenates the NumPy arrays stored in `X_train_list` and `Y_train_list` into single NumPy arrays `X_train` and `y_train`.

In [ ]:
X_train = np.concatenate(X_train_list, axis=0)
y_train = np.concatenate(Y_train_list, axis=0)

## Displaying Training Data Shapes
This code cell prints the shapes of the `X_train` and `y_train` NumPy arrays to confirm the dimensions of the training data. The output shows `(60000, 1, 28, 28)` for `X_train` (60000 images, 1 channel, 28x28 pixels) and `(60000,)` for `y_train` (60000 labels).

In [ ]:
print(X_train.shape)
print(y_train.shape)

(60000, 1, 28, 28)
(60000,)


## Converting Test Data to NumPy Arrays
This code cell iterates through the `test_loader` and converts the PyTorch image and label tensors of the test set to NumPy arrays, storing them in `X_test_list` and `Y_test_list`.

In [ ]:
X_test_list = []
Y_test_list = []
for images, labels in test_loader:
    X_test_list.append(images.numpy())
    Y_test_list.append(labels.numpy())

## Concatenating Test Data NumPy Arrays
This code cell concatenates the NumPy arrays from `X_test_list` and `Y_test_list` into `X_test` and `y_test`.

In [ ]:
X_test = np.concatenate(X_test_list, axis=0)
y_test = np.concatenate(Y_test_list, axis=0)

## Displaying Test Data Shapes
This code cell prints the shapes of the `X_test` and `y_test` NumPy arrays, showing `(10000, 1, 28, 28)` for `X_test` and `(10000,)` for `y_test`.

In [ ]:
print(X_test.shape)
print(y_test.shape)

(10000, 1, 28, 28)
(10000,)


## NumPy Conv2D Layer Implementation
This code cell defines a `Conv2D` class from scratch using NumPy.
- The `__init__` method initializes the layer with input/output channels, kernel size, stride, padding, and bias, initializing weights with random values and biases with zeros.
- The `forward` method performs the convolution operation, handling padding and calculating the output dimensions. It iterates through batches, output channels, and output height/width, applying the convolution filter and adding bias.
- The `backward` method calculates the gradients with respect to the input, weights, and biases. It uses padding and iterates through the output to backpropagate the gradients.
- The `update` method updates the weights and biases using the calculated gradients and a learning rate.

In [ ]:
class Conv2D:
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=1, bias=True):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.bias = bias

        self.weights = np.random.randn(out_channels, in_channels, kernel_size, kernel_size) * 0.1
        self.biases = np.zeros(out_channels) if bias else None

    def forward(self, input):
        self.input = input
        batch_size, in_channels, in_height, in_width = input.shape
        pad, stride, k = self.padding, self.stride, self.kernel_size

        padded_input = np.pad(input, ((0, 0), (0, 0), (pad, pad), (pad, pad)), mode='constant')
        out_height = (in_height - k + 2 * pad) // stride + 1
        out_width = (in_width - k + 2 * pad) // stride + 1

        output = np.zeros((batch_size, self.out_channels, out_height, out_width))

        for b in range(batch_size):
            for oc in range(self.out_channels):
                for i in range(out_height):
                    for j in range(out_width):
                        h_start = i * stride
                        h_end = h_start + k
                        w_start = j * stride
                        w_end = w_start + k

                        input_slice = padded_input[b, :, h_start:h_end, w_start:w_end]
                        output[b, oc, i, j] = np.sum(input_slice * self.weights[oc])
                        if self.bias:
                            output[b, oc, i, j] += self.biases[oc]
        return output

    def backward(self, d_out):
        batch_size, _, in_height, in_width = self.input.shape
        _, _, out_height, out_width = d_out.shape
        stride, pad, k = self.stride, self.padding, self.kernel_size

        d_input = np.zeros_like(self.input)
        d_weights = np.zeros_like(self.weights)
        d_biases = np.zeros_like(self.biases) if self.bias else None

        padded_input = np.pad(self.input, ((0, 0), (0, 0), (pad, pad), (pad, pad)), mode='constant')
        padded_d_input = np.pad(d_input, ((0, 0), (0, 0), (pad, pad), (pad, pad)), mode='constant')

        for b in range(batch_size):
            for oc in range(self.out_channels):
                for i in range(out_height):
                    for j in range(out_width):
                        h_start = i * stride
                        h_end = h_start + k
                        w_start = j * stride
                        w_end = w_start + k

                        input_slice = padded_input[b, :, h_start:h_end, w_start:w_end]
                        padded_d_input[b, :, h_start:h_end, w_start:w_end] += self.weights[oc] * d_out[b, oc, i, j]
                        d_weights[oc] += input_slice * d_out[b, oc, i, j]
                        if self.bias:
                            d_biases[oc] += d_out[b, oc, i, j]

        if pad > 0:
            d_input = padded_d_input[:, :, pad:-pad, pad:-pad]
        else:
            d_input = padded_d_input

        self.d_weights = d_weights
        if self.bias:
            self.d_biases = d_biases

        return d_input

    def update(self, lr):
        self.weights -= lr * self.d_weights
        if self.bias:
            self.biases -= lr * self.d_biases


## NumPy ReLu Activation Implementation
This code cell defines a `ReLu` (Rectified Linear Unit) activation function implementation using NumPy.
- The `forward` method applies the ReLu function: `max(0, input)`. It stores the input for the backward pass.
- The `backward` method calculates the gradient of the ReLu function, which is 1 for positive inputs and 0 otherwise, and multiplies it by the incoming gradient (`d_out`).

In [ ]:
class ReLu:
  def __init__(self):
    pass

  def forward(self, input):
    self.input = input
    return np.maximum(0, input)

  def backward(self, d_out):
    return d_out * (self.input > 0)

## NumPy MaxPool2D Layer Implementation
This code cell defines a `MaxPool2D` layer implementation using NumPy.
- The `__init__` method initializes the layer with kernel size and stride.
- The `forward` method performs the max pooling operation. It calculates the output dimensions and iterates through batches, channels, and output height/width, finding the maximum value within each window. It also stores a mask (`max_indices`) to keep track of the location of the maximum value for the backward pass.
- The `backward` method backpropagates the gradient through the max pooling layer. It places the incoming gradient (`d_out`) only at the locations where the maximum values were found during the forward pass, using the stored `max_indices`.

In [ ]:
class MaxPool2D:
  def __init__(self, kernel_size, stride):
    self.kernel_size = kernel_size
    self.stride = stride

  def forward(self, input):
    self.input = input
    batch_size, channels, height, width = input.shape

    out_height = (height - self.kernel_size) // self.stride + 1
    out_width = (width - self.kernel_size) // self.stride + 1

    output = np.zeros((batch_size, channels, out_height, out_width))
    self.max_indices = np.zeros_like(input, dtype=bool)




    for b in range(batch_size):
            for c in range(channels):
                for i in range(out_height):
                    for j in range(out_width):
                        h_start = i * self.stride
                        h_end = h_start + self.kernel_size
                        w_start = j * self.stride
                        w_end = w_start + self.kernel_size

                        window = input[b, c, h_start:h_end, w_start:w_end]
                        max_val = np.max(window)
                        output[b, c, i, j] = max_val

                        # Save mask for backprop
                        max_mask = (window == max_val)
                        self.max_indices[b, c, h_start:h_end, w_start:w_end] += max_mask
    return output

    def backward(self, d_out):
        d_input = np.zeros_like(self.input)
        batch_size, channels, out_height, out_width = d_out.shape

        for b in range(batch_size):
            for c in range(channels):
                for i in range(out_height):
                    for j in range(out_width):
                        h_start = i * self.stride
                        h_end = h_start + self.kernel_size
                        w_start = j * self.stride
                        w_end = w_start + self.kernel_size

                        mask = self.max_indices[b, c, h_start:h_end, w_start:w_end]
                        d_input[b, c, h_start:h_end, w_start:w_end] += mask * d_out[b, c, i, j]
        return d_input

## NumPy Flatten Layer Implementation
This code cell defines a `Flatten` layer implementation using NumPy.
- The `forward` method reshapes the input tensor into a 2D tensor, preserving the batch size and flattening all other dimensions. It stores the original input shape for the backward pass.
- The `backward` method reshapes the incoming gradient (`d_out`) back to the original input shape.

In [ ]:
class Flatten:
    def forward(self, input):
        self.input_shape = input.shape
        return input.reshape(self.input_shape[0], -1)

    def backward(self, d_out):
        return d_out.reshape(self.input_shape)

## NumPy Fully Connected Layer Implementation
This code cell defines a `Fully_Connected_Layer` (also known as a Dense or Linear layer) implementation using NumPy.
- The `__init__` method initializes the layer with input and output sizes and a learning rate. It initializes weights with random values and biases with zeros.
- The `forward` method performs the matrix multiplication of the input and weights, and adds the biases. It stores the input for the backward pass.
- The `backward` method calculates the gradients with respect to the weights, biases, and input. It also updates the weights and biases using the learning rate and calculated gradients (this update step is typically done in an optimizer, but is included here for simplicity in this scratch implementation).

In [ ]:
class Fully_Connected_Layer:
    def __init__(self, input_size, output_size, lr):
      self.weights = np.random.randn(input_size, output_size) * 0.1
      self.biases = np.zeros(output_size)
      self.lr = lr

    def forward(self, input):
      self.input = input
      return np.dot(input, self.weights) + self.biases

    def backward(self, d_out):
      self.d_weights = np.dot(self.input.T, d_out) / self.input.shape[0]
      self.d_biases = np.sum(d_out, axis=0)
      d_input = np.dot(d_out, self.weights.T)

      self.weights -= self.lr * self.d_weights
      self.biases -= self.lr * self.d_biases

      return d_input


## NumPy Softmax with Cross-Entropy Loss Implementation
This code cell defines a `SoftmaxWithCrossEntropy` class, which combines the softmax activation function and the cross-entropy loss function for numerical stability and efficient gradient calculation.
- The `forward` method calculates the softmax probabilities of the input logits and then computes the cross-entropy loss using the true labels. It applies a numerical stability trick by subtracting the maximum logit value before exponentiation. It stores the true labels and the calculated probabilities for the backward pass.
- The `backward` method calculates the gradient of the cross-entropy loss with respect to the input logits. This gradient is efficiently calculated as the difference between the predicted probabilities and a one-hot encoded version of the true labels, scaled by the batch size.

In [ ]:
class SoftmaxWithCrossEntropy:
    def forward(self, logits, labels):
        # Save for backward
        self.labels = labels

        # For numerical stability
        logits_stable = logits - np.max(logits, axis=1, keepdims=True)

        # Softmax
        exp_scores = np.exp(logits_stable)
        self.probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

        # Cross-entropy loss
        batch_size = logits.shape[0]
        correct_logprobs = -np.log(self.probs[np.arange(batch_size), labels])
        loss = np.mean(correct_logprobs)
        return loss

    def backward(self):
        # Gradient of loss w.r.t. logits
        batch_size = self.probs.shape[0]
        d_logits = self.probs.copy()
        d_logits[np.arange(batch_size), self.labels] -= 1
        d_logits /= batch_size
        return d_logits


##NumPy SimpleCNN Model Implementation
This code cell defines the `SimpleCNN` model architecture using the NumPy layers implemented from scratch.
- The `__init__` method initializes the model with a learning rate and defines a list of layers that constitute the CNN: a `Conv2D` layer, a `ReLu` activation, a `Flatten` layer, two `Fully_Connected_Layer`s with a `ReLu` in between. It also initializes the `SoftmaxWithCrossEntropy` loss function.
- The `forward` method passes the input `x` sequentially through each layer in the `self.layers` list and then calculates the loss using the `self.loss_fn`. It returns both the loss and the final output of the forward pass (the logits before the final softmax).
- The `backward` method performs the backward pass by iterating through the layers in reverse order, calling the `backward` method of each layer to compute and pass gradients backward through the network. It starts with the gradient from the loss function.
- The `update` method iterates through the layers and calls the `update` method for any layer that has one (currently, only `Fully_Connected_Layer` has an update method in this implementation, but a more complete implementation would have optimizers handling updates for all learnable layers).

In [ ]:
class SimpleCNN:
    def __init__(self, lr=0.01):
        self.layers = [
            Conv2D(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=1),
            ReLu(),
            Flatten(),
            Fully_Connected_Layer(8 * 28 * 28, 64, lr),
            ReLu(),
            Fully_Connected_Layer(64, 10, lr)
        ]
        self.loss_fn = SoftmaxWithCrossEntropy()

    def forward(self, x, y):
        for layer in self.layers:
            x = layer.forward(x)
        loss = self.loss_fn.forward(x, y)
        return loss, x

    def backward(self):
        grad = self.loss_fn.backward()
        for layer in reversed(self.layers):
            grad = layer.backward(grad)

    def update(self, lr):
        for layer in self.layers:
            if hasattr(layer, 'update'):
                layer.update(lr)


##  Training the NumPy SimpleCNN Model
This code cell trains the `SimpleCNN` model implemented in NumPy.
- It first shrinks the training dataset to 1000 samples (`X_train_small`, `y_train_small`) to speed up training, or it will take a lot of time.
- It initializes the `SimpleCNN` model, sets the number of epochs, batch size, and learning rate.
- It then enters a loop for the specified number of epochs.
- Inside the epoch loop, it shuffles the training data and iterates through it in batches.
- For each batch, it performs the forward pass to calculate the loss and outputs, then the backward pass to calculate gradients, and finally updates the model's weights and biases.
- It includes a debug print statement every 100 batches and prints the average loss and time taken for each epoch.
The output shows the training loss decreasing over the epochs, but at a slow rate, and each epoch taking a significant amount of time (~150 seconds) due to the NumPy implementation.

In [ ]:
X_train_small = X_train[:1000]
y_train_small = y_train[:1000]

model = SimpleCNN(lr=0.01)
epochs = 10
batch_size = 32
lr = 0.01

for epoch in range(epochs):
    start_time = time.time()

    permutation = np.random.permutation(X_train_small.shape[0])
    X_train_shuffled = X_train_small[permutation]
    y_train_shuffled = y_train_small[permutation]

    epoch_loss = 0
    num_batches = 0

    for i in range(0, X_train_small.shape[0], batch_size):
        x_batch = X_train_shuffled[i:i+batch_size]
        y_batch = y_train_shuffled[i:i+batch_size]

        if x_batch.ndim == 3:
            x_batch = np.expand_dims(x_batch, 1)

        loss, _ = model.forward(x_batch, y_batch)
        model.backward()
        model.update(lr)

        epoch_loss += loss
        num_batches += 1

        # Debug print every 100 batches
        if num_batches % 100 == 0:
            print(f"Epoch {epoch+1} | Batch {num_batches} done...")

    avg_loss = epoch_loss / num_batches
    end_time = time.time()

    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f} - Time: {end_time - start_time:.2f}s ✅")


Epoch 1/10 - Loss: 2.3906 - Time: 157.99s ✅
Epoch 2/10 - Loss: 2.2597 - Time: 154.27s ✅
Epoch 3/10 - Loss: 2.2275 - Time: 153.30s ✅
Epoch 4/10 - Loss: 2.2016 - Time: 153.53s ✅
Epoch 5/10 - Loss: 2.1818 - Time: 153.72s ✅
Epoch 6/10 - Loss: 2.1665 - Time: 155.72s ✅
Epoch 7/10 - Loss: 2.1541 - Time: 154.46s ✅
Epoch 8/10 - Loss: 2.1227 - Time: 154.95s ✅
Epoch 9/10 - Loss: 2.0942 - Time: 155.71s ✅
Epoch 10/10 - Loss: 2.0501 - Time: 154.25s ✅


## NumPy Model Evaluation Function
This code cell defines a function `evaluate` to evaluate the performance of the NumPy-based `SimpleCNN` model on a given dataset (typically the test set).
- It initializes counters for correct predictions, total samples, total loss, and number of batches.
- It iterates through the input data in batches.
- For each batch, it performs a forward pass to get the logits and loss.
- It calculates the predicted class by taking the argmax of the logits.
- It updates the correct and total counts, and the total loss.
- Finally, it calculates and prints the average loss and the accuracy on the provided dataset.

In [ ]:
def evaluate(model, X_test, y_test, batch_size=32):
    correct = 0
    total = 0
    total_loss = 0
    num_batches = 0

    for i in range(0, X_test.shape[0], batch_size):
        x_batch = X_test[i:i+batch_size]
        y_batch = y_test[i:i+batch_size]

        # Ensure channel dimension (Bx1x28x28)
        if x_batch.ndim == 3:
            x_batch = np.expand_dims(x_batch, 1)

        loss, logits = model.forward(x_batch, y_batch)
        preds = np.argmax(logits, axis=1)

        correct += np.sum(preds == y_batch)
        total += y_batch.shape[0]
        total_loss += loss
        num_batches += 1

    avg_loss = total_loss / num_batches
    accuracy = correct / total
    print(f"\n🧪 Test Evaluation → Loss: {avg_loss:.4f}, Accuracy: {accuracy * 100:.2f}% ✅")


##  Evaluating NumPy Model on Test Data
This code cell calls the `evaluate` function to evaluate the performance of the trained NumPy `SimpleCNN` model on the first 1000 samples of the test dataset (`X_test[:1000]`, `y_test[:1000]`).
The output shows the test loss (2.1149) and accuracy (26.80%), indicating that the NumPy model, trained on a small subset of data, is not performing well, likely due to the simplicity of the model and the dataset size used for training.

In [ ]:
evaluate(model, X_test[:1000], y_test[:1000])



🧪 Test Evaluation → Loss: 2.1149, Accuracy: 26.80% ✅


## PyTorch SimpleCNN Model Definition
This code cell defines a `SimpleCNN` model using PyTorch's `nn.Module`. This provides a comparison to the scratch implementation.
- It inherits from `nn.Module`.
- The `__init__` method defines the layers: two `nn.Conv2d` layers, a `nn.MaxPool2d` layer, and two `nn.Linear` (fully connected) layers. The kernel sizes and padding are specified for the convolutional layers, and the kernel size and stride for the pooling layer. The input size to the first fully connected layer is calculated based on the output size of the convolutional and pooling layers.
- The `forward` method defines the forward pass of the model. It applies convolutional layers followed by ReLU activation and max pooling, then flattens the output, and finally applies the fully connected layers with ReLU activation in between.

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 28x28 → 14x14
        x = self.pool(F.relu(self.conv2(x)))  # 14x14 → 7x7
        x = x.view(-1, 32 * 7 * 7)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


##  Initializing PyTorch Model, Loss Function, and Optimizer
This code cell initializes the PyTorch `SimpleCNN` model, defines the loss function, and sets up the optimizer.
- `device = torch.device("cpu")` sets the device to CPU (could be changed to "cuda" if a GPU is available).
- `model = SimpleCNN().to(device)` creates an instance of the PyTorch `SimpleCNN` model and moves it to the specified device.
- `criterion = nn.CrossEntropyLoss()` defines the cross-entropy loss function, which is commonly used for classification tasks.
- `optimizer = optim.Adam(model.parameters(), lr=0.001)` defines the Adam optimizer, which will be used to update the model's parameters during training. The learning rate is set to 0.001.

In [ ]:
device = torch.device("cpu")
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Training the PyTorch SimpleCNN Model
This code cell trains the PyTorch `SimpleCNN` model.
- It sets the number of training epochs.
- It enters a loop for the specified number of epochs.
- Inside the epoch loop, it sets the model to training mode (`model.train()`) and iterates through the `train_loader` to get batches of inputs and labels.
- It moves the inputs and labels to the specified device.
- It resets the gradients of the optimizer to zero (`optimizer.zero_grad()`).
- It performs the forward pass (`outputs = model(inputs)`), calculates the loss (`loss = criterion(outputs, labels)`), performs the backward pass (`loss.backward()`), and updates the model's parameters (`optimizer.step()`).
- It keeps track of the running loss and prints the loss every 500 batches and the average loss at the end of each epoch.
The output shows the training loss decreasing rapidly and significantly over the epochs, indicating that the PyTorch implementation is learning effectively on the full training dataset.

In [ ]:
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if (batch_idx + 1) % 500 == 0:
            print(f"[Epoch {epoch+1}] Batch {batch_idx+1} Loss: {loss.item():.4f}")

    print(f"Epoch {epoch+1} done - Average Loss: {running_loss / len(train_loader):.4f} ✅")

[Epoch 1] Batch 500 Loss: 0.1755
[Epoch 1] Batch 1000 Loss: 0.2673
[Epoch 1] Batch 1500 Loss: 0.0383
[Epoch 1] Batch 2000 Loss: 0.5179
[Epoch 1] Batch 2500 Loss: 0.0518
[Epoch 1] Batch 3000 Loss: 0.0181
[Epoch 1] Batch 3500 Loss: 0.0969
[Epoch 1] Batch 4000 Loss: 0.0295
[Epoch 1] Batch 4500 Loss: 0.0280
[Epoch 1] Batch 5000 Loss: 0.0081
[Epoch 1] Batch 5500 Loss: 0.0121
[Epoch 1] Batch 6000 Loss: 0.0297
Epoch 1 done - Average Loss: 0.1159 ✅
[Epoch 2] Batch 500 Loss: 0.0282
[Epoch 2] Batch 1000 Loss: 0.0094
[Epoch 2] Batch 1500 Loss: 0.0006
[Epoch 2] Batch 2000 Loss: 0.0052
[Epoch 2] Batch 2500 Loss: 0.0007
[Epoch 2] Batch 3000 Loss: 0.0667
[Epoch 2] Batch 3500 Loss: 0.0065
[Epoch 2] Batch 4000 Loss: 0.0062
[Epoch 2] Batch 4500 Loss: 0.0301
[Epoch 2] Batch 5000 Loss: 0.0004
[Epoch 2] Batch 5500 Loss: 0.0097
[Epoch 2] Batch 6000 Loss: 0.1804
Epoch 2 done - Average Loss: 0.0435 ✅
[Epoch 3] Batch 500 Loss: 0.0032
[Epoch 3] Batch 1000 Loss: 0.0554
[Epoch 3] Batch 1500 Loss: 0.0008
[Epoch 3]

## Evaluating PyTorch Model on Test Data
This code cell evaluates the performance of the trained PyTorch `SimpleCNN` model on the full test dataset.
- It sets the model to evaluation mode (`model.eval()`).
- It initializes counters for correct predictions and total samples.
- It uses `torch.no_grad()` to disable gradient calculation during evaluation, which saves memory and computation.
- It iterates through the `test_loader` to get batches of inputs and labels.
- It moves the inputs and labels to the specified device.
- It performs the forward pass to get the model outputs.
- It finds the predicted class for each sample using `torch.max(outputs, 1)`.
- It updates the correct and total counts.
- Finally, it calculates and prints the overall test accuracy.
The output shows a high test accuracy (98.92%), demonstrating the effectiveness of the PyTorch implementation on the MNIST dataset.

In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"\n✅ Test Accuracy: {accuracy:.2f}%")


✅ Test Accuracy: 98.92%


## Conclusion

This notebook explored the implementation of a Convolutional Neural Network (CNN) from scratch using NumPy and compared its performance to a similar model implemented using PyTorch.

### CNN Architecture Concepts

The CNN architecture implemented in this notebook, in both the NumPy and PyTorch versions, utilizes several fundamental building blocks:

-   **Convolutional Layers (Conv2D):** These layers apply a set of learnable filters (kernels) to the input image. Each filter slides over the input, performing element-wise multiplications and summing the results to create a feature map. This process helps in detecting spatial hierarchies of features, such as edges, corners, and textures. The NumPy implementation (`Conv2D`) manually handles padding and strides, while the PyTorch implementation (`nn.Conv2d`) provides these functionalities directly.

-   **Activation Functions (ReLU):** Rectified Linear Unit (ReLU) is a non-linear activation function that introduces non-linearity into the network. It outputs the input directly if it's positive, and zero otherwise (`max(0, input)`). This non-linearity allows the network to learn more complex patterns than a linear model. Both implementations (`ReLu` in NumPy and `F.relu` in PyTorch) apply this function element-wise.

-   **Pooling Layers (MaxPool2D):** Pooling layers reduce the spatial dimensions (width and height) of the feature maps while retaining the most important information. Max pooling, as used here, selects the maximum value within a defined window (kernel size) as it slides over the feature map. This helps to reduce the number of parameters and computation, and also provides some spatial invariance. The NumPy implementation (`MaxPool2D`) manually tracks the indices of the maximum values for backpropagation, while PyTorch's `nn.MaxPool2d` handles this automatically.

-   **Flatten Layer:** This layer reshapes the multi-dimensional output of the convolutional and pooling layers into a one-dimensional vector. This is necessary before feeding the output into a fully connected layer. Both the NumPy (`Flatten`) and PyTorch (`x.view(-1, ...)`) implementations perform this reshaping.

-   **Fully Connected Layers (Dense/Linear):** These layers are standard neural network layers where each neuron is connected to every neuron in the previous layer. They take the flattened feature vector as input and perform a linear transformation (matrix multiplication with weights and adding biases) followed by an activation function. The NumPy implementation (`Fully_Connected_Layer`) includes the weight and bias updates within the layer's backward method, while in PyTorch (`nn.Linear`), the optimization is handled separately by an optimizer.

-   **Softmax:** The softmax function is typically applied to the output of the final fully connected layer in a classification task. It converts the raw output scores (logits) into probabilities that sum up to 1. Each value in the output represents the probability of the input belonging to a particular class. In the NumPy implementation, this is part of the `SoftmaxWithCrossEntropy` class's forward pass.

-   **Loss Function (Cross-Entropy Loss):** Cross-entropy loss is a common loss function for multi-class classification problems. It measures the difference between the predicted probabilities (from the softmax layer) and the true class labels. The goal of training is to minimize this loss. The NumPy implementation calculates this within the `SoftmaxWithCrossEntropy` class, and PyTorch uses `nn.CrossEntropyLoss`.

### Comparison of NumPy and PyTorch Models

The notebook clearly demonstrates the differences in performance and training efficiency between the NumPy-based scratch implementation and the PyTorch implementation:

-   **Training Data Size:** The NumPy model was trained on a small subset of the training data (1000 samples) due to the significantly slower execution speed of the manual implementation. The PyTorch model was trained on the full training dataset (60000 samples).

-   **Training Loss:** The training loss for the NumPy model decreased slowly over 10 epochs, ending at a loss of approximately 2.05. The training loss for the PyTorch model decreased much more rapidly and significantly over just 5 epochs, reaching a much lower average loss of approximately 0.016. This highlights the computational efficiency and optimization capabilities of PyTorch.

-   **Training Time:** Each epoch of the NumPy model training took around 150 seconds, even with a small subset of data. The PyTorch model trained on the full dataset completed 5 epochs in a relatively short amount of time (around 10 minutes based on the timestamps), showcasing the power of optimized libraries and potentially GPU acceleration (although it was run on CPU in this case).

-   **Test Performance:** The NumPy model evaluated on 1000 test samples achieved a low accuracy of 26.80% and a test loss of 2.1149. The PyTorch model evaluated on the full test dataset achieved a high accuracy of 98.92%. This significant difference in performance is primarily due to training on the full dataset and the optimized nature of the PyTorch framework.

-   **Loss Functions:** Both implementations use the concept of cross-entropy loss. The NumPy version implements it manually within the `SoftmaxWithCrossEntropy` class, including the softmax calculation for numerical stability. The PyTorch version uses the built-in and highly optimized `nn.CrossEntropyLoss`.

In summary, while the NumPy implementation provided valuable insight into the inner workings of each layer and the backpropagation process, it is significantly less efficient for practical deep learning tasks compared to a framework like PyTorch, which leverages optimized operations and simplifies the development process. The PyTorch model achieved a much higher accuracy on the MNIST dataset due to training on the full dataset and the inherent advantages of the framework.